<a href="https://colab.research.google.com/github/sunmarknatecom/01_markcv/blob/master/ct_segmentation_v04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA PREPROCESSING

In [2]:
!pip install pydicom

     |████████████████████████████████| 2.0 MB 13.8 MB/s 


# Import the modules.

## Configure the parameters.

In [3]:
root = '/content/drive/MyDrive/segmentation/data/'
idx_path = 'seg_006/'
data_path = ['dicom/','image/','mask/','prediction/']
train_idx = 'seg_001/'

## Define the functions.

In [3]:
from glob import glob
import numpy as np
import pydicom
import cv2
import os
from pydicom.pixel_data_handlers.util import apply_modality_lut, apply_voi_lut
import matplotlib.pyplot as plt

import tensorflow as tf

from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, BatchNormalization, concatenate
from keras.models import Model
from tensorflow.keras.optimizers import Adam

from keras import backend as K
from keras.losses import binary_crossentropy

root = '/content/drive/MyDrive/segmentation/data/'
idx_path = 'seg_006/'
data_path = ['dicom/','image/','mask/','prediction/']

class Annot_Mk():
    '''
    The class for segmentation
    STEP1: transform the dicom file to jpg image file
    '''
    def __init__(self):
        self.root_path = '/content/drive/MyDrive/segmentation/data/'
        self.idx_path = 'seg_006/'
        self.data_path = ['dicom/','image/','mask/','prediction/']
        self.window_center = -1400
        self.window_width = 1300
    # def dcm_to_img(src_img):
    #     '''
    #     src(1 param): dicom pixel nd array
    #     return : float array [0-1]
    #     requried package: numpy
    #     cv2.normalize(src, dst[, alpha[, beta[, norm_type[, dtype[, mask]]]]]	) ->	dst
    #     ex) cv2.normalize(src_img, None, 0, 255, cv2.NORM_MINMAX)
    #     ref) https://docs.opencv.org/3.4/d2/de8/group__core__array.html#ga87eef7ee3970f86906d69a92cbf064bd
    #     DEPRICATED...
    #     '''
    #     MAX_VAL = np.max(src_img)
    #     MIN_VAL = np.min(src_img)
    #     U_DIV = src_img - MIN_VAL
    #     D_DIV = MAX_VAL - MIN_VAL
    #     temp_img = U_DIV / D_DIV
    #     scaled_img = 255 * temp_img
    #     ret_img = np.array(scaled_img, dtype='uint8')
    #     return ret_img
    def raw_HU(self, path):
        slice = pydicom.dcmread(path)
        image = slice.pixel_array
        return image
    def set_HU(self, path, wnd_c = -1400, wnd_w = 1300):
        '''
        src(3 params): path is dcm file name, window_center(wnd_c), window_width(wnd_w)
        return: window adjusted pixel_array
        '''
        slice = pydicom.dcmread(path)
        b = int(slice.RescaleIntercept)
        image = int(slice.RescaleSlope) * slice.pixel_array + b
        slice.WindowCenter = wnd_c
        slice.WindowWidth = wnd_w
        image = apply_modality_lut(image, slice)
        image2 = apply_voi_lut(image, slice)
        image3 = np.clip(image2, wnd_c - (wnd_w/2), wnd_c + (wnd_w/2))
        return image3
    def dcm2jpg(self, wnd_c = -1400, wnd_w = 1300, wnd_o = True):
        '''
        USAGE:
        C:>python dcm_to_jpg.py
        '''
        work_path = self.root_path + self.idx_path
        if not os.path.exists(work_path + self.data_path[1]):
            os.mkdir(work_path + self.data_path[1])  
        LIST_FILES = sorted(glob(work_path + self.data_path[0] + "*.dcm"))
        if wnd_o == True:
            for i, elem in enumerate(LIST_FILES):
                OBJ_ARR_WND = self.set_HU(path=elem, wnd_c=wnd_c, wnd_w=wnd_w)
                # OBJ_ARRAY = dtj_cvt.dcm_to_img(OBJ_PIXEL)
                OBJ_ARR_NRM = cv2.normalize(OBJ_ARR_WND, None, 0, 255, cv2.NORM_MINMAX)
                OBJ_temp = OBJ_ARR_NRM.astype(np.uint8)
                OBJ_IMG = cv2.cvtColor(OBJ_temp, cv2.COLOR_GRAY2RGB)
                cv2.imwrite(work_path + self.data_path[1] + elem[-9:-3]+'jpg', OBJ_IMG)
                progress = ("]]] %3d" %(int(round(100*i/len(LIST_FILES))))) + "% completed."
                print(progress, end='\r')
            print(progress)
        elif wnd_o == False:
            for i, elem in enumerate(LIST_FILES):
                OBJ_ARR_WND = self.raw_HU(path=elem)
                # OBJ_ARRAY = dtj_cvt.dcm_to_img(OBJ_PIXEL)
                OBJ_ARR_NRM = cv2.normalize(OBJ_ARR_WND, None, 0, 255, cv2.NORM_MINMAX)
                OBJ_temp = OBJ_ARR_NRM.astype(np.uint8)
                OBJ_IMG = cv2.cvtColor(OBJ_temp, cv2.COLOR_GRAY2RGB)
                cv2.imwrite(work_path + self.data_path[1] + elem[-9:-3]+'jpg', OBJ_IMG)
                progress = ("]]] %3d" %(int(round(100*i/len(LIST_FILES))))) + "% completed."
                print(progress, end='\r')
            print(progress)
        else:
            pass
    def input_img_data(self):
        input_path = self.root_path + self.idx_path + self.data_path[1]
        train_img_paths = sorted(glob(input_path +'*.jpg'))
        train_imgs = np.array([cv2.resize(cv2.imread(path, cv2.IMREAD_UNCHANGED), (512,512)) for path in train_img_paths])
        return train_imgs
    def input_msk_data(self):
        input_path = self.root_path + self.idx_path + self.data_path[2]
        train_msk_paths = sorted(glob(input_path+'*.png'))
        train_msks = np.array([cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (512,512)) for path in train_msk_paths])
        train_msks = train_msks.astype(np.float32)
        train_msks = np.reshape(train_msks, (*train_msks.shape, 1))
        train_msks -= 1
        return train_msks
    def unet_training(self):
        inputs = Input((512, 512, 3))
        bnorm1 = BatchNormalization()(inputs)
        conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(bnorm1)
        conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
        conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
        conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
        conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
        conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
        conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
        up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
        conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
        conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
        up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
        conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
        conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
        up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
        conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
        conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
        up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
        conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
        conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
        conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
        model = Model(inputs=[inputs], outputs=[conv10])
        SMOOTH = 1.
        def dice_coef(y_true, y_pred):
            y_true_f = K.flatten(y_true)
            y_pred_f = K.flatten(y_pred)
            intersection = K.sum(y_true_f * y_pred_f)
            return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)
        def bce_dice_loss(y_true, y_pred):
            return 0.5 * binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)
        model.compile(Adam(learning_rate=1e-4), bce_dice_loss, metrics=[binary_crossentropy, dice_coef])
        model.fit(train_imgs[50:], train_msks[50:], batch_size=12, epochs=5, validation_data=(train_imgs[:50], train_msks[:50]))

In [4]:
obj = Annot_Mk()

In [6]:
obj.dcm2jpg(wnd_o=True)


]]] 100% completed.


In [25]:
list = sorted(os.listdir('/content/drive/MyDrive/segmentation/data/seg_006/image'))

In [ ]:
list

In [33]:
img2 = cv2.imread(+list[0], cv2.IMREAD_UNCHANGED)

In [ ]:
img2

In [ ]:
list = sorted(os.listdir('/content/drive/MyDrive/segmentation/data/seg_006/image'))
for elem in list:
    img = cv2.imread('/content/drive/MyDrive/segmentation/data/seg_006/image/'+elem, cv2.IMREAD_UNCHANGED)
    plt.imshow(img, cmap='gray')
    plt.show()

## Confirm the GPU.

In [ ]:
!nvidia-smi
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

In [ ]:
train_img_paths = sorted(glob(root + train_idx + data_path[1] + '*.jpg'))
train_msk_paths = sorted(glob(root + train_idx + data_path[2] + '*.png'))

In [ ]:
train_imgs = np.array([cv2.resize(cv2.imread(path, cv2.IMREAD_UNCHANGED), (512,512)) for path in train_img_paths])
train_msks = np.array([cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (512,512)) for path in train_msk_paths])

train_msks = train_msks.astype(np.float32)
train_msks = np.reshape(train_msks, (*train_msks.shape, 1))

In [ ]:
train_msks -= 1

In [ ]:
%matplotlib inline
fig = plt.figure(0, figsize=(20, 20))
fig.add_subplot(1, 2, 1)
plt.imshow(train_imgs[0])
fig.add_subplot(1, 2, 2)
plt.imshow(np.squeeze(train_msks[0]), cmap='gray')

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, BatchNormalization, concatenate
from keras.models import Model


inputs = Input((512, 512, 3))
bnorm1 = BatchNormalization()(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(bnorm1)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])

In [ ]:
from keras import backend as K
from keras.losses import binary_crossentropy

SMOOTH = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def bce_dice_loss(y_true, y_pred):
    return 0.5 * binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(Adam(learning_rate=1e-4), bce_dice_loss, metrics=[binary_crossentropy, dice_coef])

model.fit(train_imgs[50:], train_msks[50:], batch_size=12, epochs=5, validation_data=(train_imgs[:50], train_msks[:50]))

In [ ]:
file_list = sorted(glob(root + idx_path + data_path[1] + '*.jpg'))
predic = []

for elem in file_list:
    img_obj = cv2.imread(elem, cv2.IMREAD_UNCHANGED)
    predic.append(img_obj)

prediction = np.array(predic)

In [ ]:
pred_img = model.predict(prediction)

In [ ]:
temp_pred = []

for elem in pred_img:
    bgr_img = cv2.cvtColor(elem, cv2.COLOR_GRAY2RGB)
    temp_pred.append(bgr_img)

In [ ]:
for i, img in enumerate(temp_pred):
    path_ = '/content/drive/MyDrive/segmentation/data/seg_006/mask/mask_1-%03d.png'%(i+1)
    img = img * 255
    cv2.imwrite(path_, img)
    plt.imshow(img)
    plt.show()

In [ ]:
temp_2_pred = []
for elem in temp_pred:
    elem[elem<0.5] = 0
    elem[elem>=0.5] = 1
    elem.astype('uint8')
    temp_2_pred.append(elem)

In [ ]:
for i, elem in enumerate(temp_2_pred):
    path = '/content/drive/MyDrive/Seg_Data/result/result_%03d.png'%(i+1)
    cv2.imwrite(path, elem)